In [1]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

def generate_tags(question):
    # Preprocess the question
    encoded_text = tokenizer(question, return_tensors="pt")

    # Generate tags using the pretrained model
    with tf.device("/gpu:0" if tf.test.is_gpu_available() else "/cpu:0"):
        outputs = model(**encoded_text)
    logits = outputs.logits.detach().cpu().numpy()

    # Get the top 5 tags
    top_5_tags = logits.argsort(axis=1)[:, -5:]

    # Convert tag indices to tag names
    tags_list = [[tokenizer.convert_ids_to_tokens(tags_idx)] for tags_idx in top_5_tags]


    return tags_list


# Example usage
question = """What is the use of the yield keyword in Python? What does it do?

For example, I'm trying to understand this code1:

def _get_child_candidates(self, distance, min_dist, max_dist):
    if self._leftchild and distance - max_dist < self._median:
        yield self._leftchild
    if self._rightchild and distance + max_dist >= self._median:
        yield self._rightchild  
And this is the caller:

result, candidates = [], [self]
while candidates:
    node = candidates.pop()
    distance = node._get_dist(obj)
    if distance <= max_dist and distance >= min_dist:
        result.extend(node._values)
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))
return result
What happens when the method _get_child_candidates is called? Is a list returned? A single element? Is it called again? When will subsequent calls stop?"""
tags = generate_tags(question)
print(tags)


2023-11-16 19:18:52.243421: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 19:18:52.283029: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 19:18:52.490759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 19:18:52.490938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 19:18:52.522157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[[['[PAD]', '[unused0]', '[unused2]', '[unused1]', '[unused3]']]]
